In [1]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue
from datetime import datetime

In [2]:
def reexecute_and_train(func_name, cells):
    for cell in cells[:-1]:
        exec(cell, globals())
    
    filename = f"model-{datetime.now().isoformat()}"
    globals()[func_name] = globals()[func_name].original
    exec(cells[-1], globals())
    torch.save(model.state_dict(), filename)
    return filename

In [3]:
def execute_in_subprocess():
    def reexecute_and_train_in_subprocess(func_name, cells, queue):
        filename = reexecute_and_train(func_name, cells)
        queue.put(filename)
    
    queue = Queue()
    p = Process(target=reexecute_and_train_in_subprocess, args=(func_name, cells, queue))
    p.start()
    p.join() # this blocks until the process terminates
    filename = queue.get()
    return filename

In [4]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue

address = ('localhost', 6000)     # family is deduced to be 'AF_INET'
listener = Listener(address, authkey=b'secret password')
try:
    while True:
        conn = listener.accept()
        func_name= conn.recv()
        cells = conn.recv()
        # filename = reexecute_and_train(func_name, cells)
        filename = execute_in_subprocess()

        conn.send(filename)
finally:
    listener.close()

batch size: torch.Size([32, 1, 28, 28])
torch.Size([32, 10])
Epoch: 0 | Loss: 1.5536 | Train Accuracy: 91.14
Epoch: 1 | Loss: 1.4923 | Train Accuracy: 97.19
batch size: torch.Size([32, 1, 28, 28])
torch.Size([32, 10])
Epoch: 0 | Loss: 2.3602 | Train Accuracy: 10.23
Epoch: 1 | Loss: 2.3602 | Train Accuracy: 10.22
Epoch: 2 | Loss: 2.3602 | Train Accuracy: 10.22
batch size: torch.Size([32, 1, 28, 28])
torch.Size([32, 10])
Epoch: 0 | Loss: 1.6337 | Train Accuracy: 83.02
Epoch: 1 | Loss: 1.4929 | Train Accuracy: 97.19
Epoch: 2 | Loss: 1.4815 | Train Accuracy: 98.21
Epoch: 3 | Loss: 1.4769 | Train Accuracy: 98.66
Epoch: 4 | Loss: 1.4740 | Train Accuracy: 98.92
batch size: torch.Size([32, 1, 28, 28])
torch.Size([32, 10])
Test Accuracy: 10.16


Process Process-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_376234/536911192.py", line 3, in reexecute_and_train_in_subprocess
    filename = reexecute_and_train(func_name, cells)
  File "/tmp/ipykernel_376234/1347668482.py", line 3, in reexecute_and_train
    exec(cell, globals())
  File "<string>", line 1, in <module>
TypeError: 'builtin_function_or_method' object is not subscriptable
